In [3]:
import io
import numpy as np
import torch.onnx
import torch.utils.model_zoo as model_zoo
import torch.nn.init as init
from torch import nn

In [4]:
#Super Resolution model definition in Pytorch
class SuperResolutionNet(nn.Module):
    def __init__(self,upscale_factor,inplace=False):
        super(SuperResolutionNet,self).__init__()
        self.relu=nn.ReLU(inplace=inplace)
        self.conv1=nn.Conv2d(1,64,(5,5),(1,1),(2,2))
        self.conv2=nn.Conv2d(64,64,(3,3),(1,1),(1,1))
        self.conv3=nn.Conv2d(64,32,(3,3),(1,1),(1,1))
        self.conv4=nn.Conv2d(32,upscale_factor**2,(3,3),(1,1),(1,1))
        self.pixel_shuffle=nn.PixelShuffle(upscale_factor)
        self._initialize_weights()
    def forward(self,x):
        x=self.relu(self.conv1(x))
        x=self.relu(self.conv2(x))
        x=self.relu(self.conv3(x))
        x=self.pixel_shuffle(self.conv4(x))
        return x
    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight,init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight,init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight,init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)
torch_model=SuperResolutionNet(upscale_factor=3)
print(torch_model)

SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pixel_shuffle): PixelShuffle(upscale_factor=3)
)


In [5]:
#Load pretrained model weights
model_url='https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size=1
#Initialize model with pretrained weights
map_location=lambda storage,loc:storage
if torch.cuda.is_available():
    map_location=None
torch_model.load_state_dict(model_zoo.load_url(model_url,map_location=map_location))
#Set the train mode False 只运行前向传播
torch_model.train(False)

#Model input
x=torch.randn(batch_size,1,224,224,requires_grad=True)
#Export model，运行模型之后的输出
torch_out=torch.onnx._export(torch_model,   #运行模型
                            x,             #Model input(或者含有多个输入的元组)
                            'super_resolution.onnx',   #保存模型的位置
                            export_params=True)       #保存模型的参数权重

Downloading: "https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth" to C:\Users\13051/.torch\models\superres_epoch100-44c6958e.pth
100.0%


In [7]:
#在caffe2中使用导出的onnx模型。证明在caffe2和Pytorch中计算的相同的网络
import onnx
import caffe2.python.onnx.backend as onnx_caffe2_backend


ImportError: cannot import name 'caffe2_pb2'